In [19]:
# -*- coding: utf-8 -*-
import pandas as pd
from pandas_profiling import ProfileReport
import json
import numpy as np
import seaborn as sns
import pdfkit
sns.set(font="cmr10")
from seaborn import heatmap
from pprint import pprint as p
import os
import sys
import subprocess
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
rcParams.update({'font.family': 'cmr10'})
import sys

# Load data
fn = 'tweets_01-08-2021.json'

with open(fn, encoding='utf-8') as f:
    df = pd.read_json(f)

# Drop before Trump ran for President.
is_president_or_running_for_president = df['date'] < '2015-06-16'
df = df.loc[np.invert(is_president_or_running_for_president)]

# Sort by RT
df.sort_values(by=['retweets'], ascending=False, inplace=True)
p(df.info())
p(df.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24564 entries, 11759 to 9906
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         24564 non-null  int64         
 1   text       24564 non-null  object        
 2   isDeleted  24564 non-null  object        
 3   device     24564 non-null  object        
 4   favorites  24564 non-null  int64         
 5   retweets   24564 non-null  int64         
 6   date       24564 non-null  datetime64[ns]
 7   isFlagged  24564 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 1.7+ MB
None
                        id                                               text  \
11759  1311892190680014800  Tonight, @FLOTUS and I tested positive for COV...   
35620   881503147168071700         #FraudNewsCNN #FNN https://t.co/WYUnHjjUjg   
39347   795954831718498300                 TODAY WE MAKE AMERICA GREAT AGAIN!   
53251  11573456925176

In [21]:
# Initialize new columns to false
with open("feature_whitelist.json") as f:
    features = list(json.load(f))

# Initialize new columns to false
df[features] = 'f'

feature_list = ['maga', 'make america great again']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature)
    is_magazine = df['text'].str.lower().str.contains('magazine')
    df.loc[is_maga, 'isMAGA'] = 't'
    df.loc[is_magazine, 'isMAGA'] = 'f'
print('isMAGA: ' + str(df['isMAGA'].str.contains('t').sum()))

# Populate witch hunt column
feature_list = ['witch hunt']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature)
    df.loc[is_maga, "isWitchHunt"] = 't'
print('isWitchHunt: ' + str(df['isWitchHunt'].str.contains('t').sum()))

# Populate in two weeks column
feature_list = ['two weeks', 'next week']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature)
    df.loc[is_maga, 'isInTwoWeeks'] = 't'
print('isInTwoWeeks: ' + str(df['isInTwoWeeks'].str.contains('t').sum()))

# Populate is so true column
feature_list = ['true.', 'so true']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature)
    df.loc[is_maga, 'isSoTrue'] = 't'
print('isSoTrue: ' + str(df['isSoTrue'].str.contains('t').sum()))

# Populate all caps column
for feature in feature_list:
    is_maga = df['text'].str.isupper()
    df.loc[is_maga, 'isAllCaps'] = 't'
print('isAllCaps: ' + str(df['isAllCaps'].str.contains('t').sum()))

# Populate is Russia column
feature_list = ['russia', 'russian', 'putin']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature)
    df.loc[is_maga, 'isRussia'] = 't'
print('isRussia: ' + str(df['isRussia'].str.contains('t').sum()))

# Populate is Ukraine column
feature_list = ['ukraine', 'ukrainian']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isUkraine'] = 't'
print('isUkraine: ' + str(df['isUkraine'].str.contains('t').sum()))

# Populate is china column
feature_list = ['china', 'chinese']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isChina'] = 't'
print('isChina: ' + str(df['isChina'].str.contains('t').sum()))

# Populate is iran column
feature_list = ['iran', 'iranian']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isIran'] = 't'
print('isIran: ' + str(df['isIran'].str.contains('t').sum()))

# Populate is antifa column
feature_list = ['antifa']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isAntifa'] = 't'
print('isAntifa: ' + str(df['isAntifa'].str.contains('t').sum()))

# Populate is love column
feature_list = ['love', 'heartwarming']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isLove'] = 't'
print('isLove: ' + str(df['isLove'].str.contains('t').sum()))

# Populate is love column
feature_list = ['fake news', 'lamestream media', 'mainstream media']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isLügenpresse'] = 't'
print('isLügenpresse (fake news, lametream media): ' + str(df['isLügenpresse'].str.contains('t').sum()))

# Populate is communism column
feature_list = ['communist', 'communism', 'marxist']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isCommunism'] = 't'
print('isCommunism: ' + str(df['isCommunism'].str.contains('t').sum()))

# Populate is capital raid column
feature_list = ['january 6th', 'jan 6th']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isCapitalRaid'] = 't'
print('isCapitalRaid: ' + str(df['isCapitalRaid'].str.contains('t').sum()))

# Populate is nukes column
feature_list = ['nuclear', 'nuke', 'nukes']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isNukes'] = 't'
print('isNukes: ' + str(df['isNukes'].str.contains('t').sum()))

# Populate is executive privilege column
feature_list = ['rino', 'republican in name only']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isRINO'] = 't'
print('isRINO: ' + str(df['isRINO'].str.contains('t').sum()))

# Populate is religious column
feature_list = ['religion', 'god', 'prayer', 'pray', 'christmas']
for feature in feature_list:
    is_maga = df['text'].str.lower().str.contains(feature) 
    df.loc[is_maga, 'isReligious'] = 't'
print('isReligious: ' + str(df['isReligious'].str.contains('t').sum()))

isMAGA: 1011
isWitchHunt: 358
isInTwoWeeks: 61
isSoTrue: 356
isAllCaps: 301
isRussia: 512
isUkraine: 152
isChina: 572
isIran: 217
isAntifa: 28
isLove: 739
isLügenpresse (fake news, lametream media): 962
isCommunism: 16
isCapitalRaid: 8
isNukes: 104
isRINO: 53
isReligious: 343


In [22]:
# Load labels from more_features.json

with open('more_features.json', encoding='utf-8') as f:
    feature_dict = json.load(f)

for k, v in feature_dict.items():
    idx = df.loc[df['id'] == int(k)].index
    
    # Value for this tweet ID is a dict {feature_1: [ref_1, ref_2, ...], ...}
    for feature in list(v.keys()):
        df.loc[idx, feature] = 't'

labeled_tweets = [int(x) for x in list(feature_dict.keys())]
label_df = df.loc[df['id'].isin(labeled_tweets)].copy()

print('isLie: ' + str(df['isLie'].str.contains('t').sum()))
print('isSmear: ' + str(df['isSmear'].str.contains('t').sum()))
print('isDolchstoßlegende (The Big Lie): ' + str((df['isDolchstoßlegende'].str.contains('t')).sum()))
print(label_df.info())

{'1311892190680014800': ['isPandemic'], '881503147168071700': ['isLügenpresse', 'isSmear'], '795954831718498300': ['isMAGA'], '1157345692517634000': ['isCelebrity'], '929511061954297900': ['isSmear', 'isBullying', 'isNorthKorea', 'isLie'], '1267129644228247600': ['isIronic', 'isAntifa', 'isHitler'], '1325099845045072000': ['isElection', 'isLie', 'isDolchstoßlegende'], '1326926226888544300': ['isElection', 'isLie', 'isDolchstoßlegende'], '1152307567634391000': ['isCelebrity'], '1199718185865535500': [], '1341537886315950000': [], '796315640307060700': [], '1323534663453913000': ['isElection'], '1325195021339988000': ['isElection'], '1346912780700577800': ['isElection', 'isLie', 'isOpposite', 'isIKnowYouAreButWhatAmI', 'isReichstagsgebäude'], '1347334804052844500': [], '1212924762827047000': [], '948355557022421000': ['isPentagon', 'isThreatening', 'isBullying', 'isWarPowersAct1973', 'isNorthKorea'], '1312525833505058800': ['isChina'], '1214739853025394700': ['isPentagon', 'isIran'], '13

isLie: 191
isSmear: 123
isDolchstoßlegende (The Big Lie): 152
<class 'pandas.core.frame.DataFrame'>
Int64Index: 506 entries, 11759 to 306
Data columns (total 65 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       506 non-null    int64         
 1   text                     506 non-null    object        
 2   isDeleted                506 non-null    object        
 3   device                   506 non-null    object        
 4   favorites                506 non-null    int64         
 5   retweets                 506 non-null    int64         
 6   date                     506 non-null    datetime64[ns]
 7   isFlagged                506 non-null    object        
 8   isLie                    506 non-null    object        
 9   isMisspeaking            506 non-null    object        
 10  isCantBeThatDumb         506 non-null    object        
 11  isOpposite               506 no

In [23]:
# Retain text, id, date, device
text_df = label_df.loc[:, ['id', 'text', 'date', 'device']]

# Drop columns not represented in the data, except twitter columns.
twitter_columns = [
    'isRetweet', 
    'isDeleted', 
    'favorites', 
    'retweets', 
    'isFlagged'
]

feature_list = []
for k, v in feature_dict.items():
    for feature in v:
        if isinstance(feature, dict):
            feature_list.append(list(feature.items())[0][0])
        else:
            feature_list.append(feature)

valid_features = list(set(feature_list).intersection(set(features)) | set(twitter_columns))

# Drop invalid columns
drop_columns = set(label_df.columns).difference(set(valid_features))
label_df.drop(list(drop_columns), axis=1, inplace=True)
print(label_df.columns.values)

['isDeleted' 'favorites' 'retweets' 'isFlagged' 'isLie' 'isOpposite'
 'isInsinuating' 'isPreemptive' 'isElection' 'isIKnowYouAreButWhatAmI'
 'isRacist' 'isHitler' 'isRussia' 'isUkraine' 'isChina' 'isNorthKorea'
 'isIran' 'isExecutivePrivilege' 'isSmear' 'isCelebrity' 'isPentagon'
 'isNickname' 'isXenophobic' 'isMAGA' 'isPandemic' 'isAllCaps' 'isRINO'
 'isFirstImpeachment' 'isAntifa' 'isBlackLivesMatter' 'isLügenpresse'
 'isDolchstoßlegende' 'isReichstagsgebäude' 'isBullying' 'isThreatening'
 'isWarPowersAct1973' 'isIronic']


In [24]:
# Convert T/F to binary
def map_fn(tf_str):
    if tf_str == 't':
        return 1
    else:
        if tf_str == 'f':
            return 0
        else:
            return tf_str
    
for col in label_df:
    label_df.loc[:, col] = label_df[col].map(map_fn)

# Enforce at least three samples
is_at_least_three_samples = label_df.sum(axis=0).values >= 3
label_df = label_df.iloc[:, np.arange(len(is_at_least_three_samples))[is_at_least_three_samples]]

# Enforce that isDolchstoßlegende implies election lie
label_df.loc[label_df.isDolchstoßlegende == 1, ['isElection', 'isLie']] = 1

# Give Hitler points for triggers
isHitler = np.zeros((len(label_df), ))
if 'isAntifa' in label_df.columns.values:
    isHitler += label_df.isAntifa
if 'isLügenpresse' in label_df.columns.values:
    isHitler += label_df.isLügenpresse
if 'isDolchstoßlegende' in label_df.columns.values:
    isHitler += label_df.isDolchstoßlegende
if 'isBlackLivesMatter' in label_df.columns.values:
    isHitler += label_df.isBlackLivesMatter
label_df.loc[:, "isHitler"] = isHitler

# TODO: Define isHitlerBingo as achieving all four Hitler dimensions in a
# single tweet. This will add an addional Hitler point to the score. 

label_df.head()

,isDeleted,favorites,retweets,isFlagged,isLie,isPreemptive,isElection,isIKnowYouAreButWhatAmI,isRacist,isRussia,...,isRINO,isAntifa,isBlackLivesMatter,isLügenpresse,isDolchstoßlegende,isReichstagsgebäude,isBullying,isThreatening,isWarPowersAct1973,isHitler
11759,0,1869706,408866,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
35620,0,503063,293109,0,0.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1.0
39347,0,498035,281289,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
53251,0,821423,226235,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
43649,0,534338,217199,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.0


In [25]:
profile = ProfileReport(label_df, title="Labeled Trump Tweets")
profile.to_file("label_profile_report.html")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

RuntimeError: latex was not able to process the following string:
b'df_index'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(/home/matt/.cache/matplotlib/tex.cache/9a7ac17a27a1d6373f6a220582baa310.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic

In [ ]:
# LaTeX builder for correlation matrix/graph. 
import shutil
import glob

try:
    os.remove('tex/main.pdf')
except:
    pass

# Compute stats and flag correlations @ p < .05
NUM_SAMPLES = 10000
SIGNIFICANCE = .95

samples = np.zeros((NUM_SAMPLES, len(label_df.columns.values)**2))
for i in range(NUM_SAMPLES):                   
    samples[i, :] = label_df.sample(n=len(label_df), replace=True).corr(min_periods=0).fillna(0).values.flatten()

# Compute significance of corr matrix
positive_sig = np.percentile(samples, 100*(1 - SIGNIFICANCE)/2, axis=0) > 0
negative_sig = np.percentile(samples, 100*(1 - (1 - SIGNIFICANCE)/2), axis=0) < 0
corr_matrix_significance = np.reshape(np.logical_or(positive_sig, negative_sig), (len(label_df.columns.values), len(label_df.columns.values)))

# Layout radial
features = label_df.columns.values
feature_range = np.array(range(0, len(features)))
layout_rad = np.arange(0.0, 2*np.pi, 2*np.pi/len(label_df.columns.values))
xshift = np.cos(layout_rad)*6.5
yshift = np.sin(layout_rad)*6.5

# Compute final correlation Matrix
corr_matrix = corr_matrix_significance.astype(float)*np.reshape(samples.mean(axis=0), (len(features), len(features)))
drop_idx = np.diag(corr_matrix) == 0.0
keep_idx = feature_range[np.invert(np.isin(feature_range, feature_range[drop_idx]))]
drop_matrix = corr_matrix[keep_idx, :][:, keep_idx]
drop_col = features[drop_idx]
keep_col = features[keep_idx]

# Add space
space_list = ['isAntifa', 'isIKnowYouAreButWhatAmI', 'isLügenpresse']
for i, feature in enumerate(features):
    if feature in space_list:
        xshift[i] = np.cos(layout_rad[i])*7.0
        yshift[i] = np.sin(layout_rad[i])*7.0

# Plot figure
plt.figure(figsize=(8.5, 8.5))
plt.rc('text', usetex=True)
corr_heatmap = heatmap(
    drop_matrix, 
    vmin=-1.0, 
    vmax=1.0, 
    center=0.0,
    square=True,
    cmap = 'RdBu',
    xticklabels=keep_col, 
    yticklabels=keep_col
)
plt.rc('axes', unicode_minus=False)
plt.savefig('tex/corr_heatmap.pdf')
plt.gcf().set_visible(False)

# Graph view
# ticks = [x.replace('ß', '\\ss ').replace('ü', '{\\"u}') for x in label_df.columns.values]
graph = '\\begin{tikzpicture}\n'
for x, y, feature in zip(xshift, yshift, features):
    graph += '\\node[const] ({feature}) [xshift={x:.15f}cm, yshift={y:.15f}cm, minimum size=1.5cm] {{{feature}}}; '.format(x=x, y=y, feature=feature)

for i in range(corr_matrix.shape[0]):
    for j in range(i):
        if corr_matrix_significance[i, j]:
            corr = corr_matrix[i, j]
            
            if corr < 0:
                color = 'red!{corr}!white'.format(corr=corr*-100)
            else:
                color = 'blue!{corr}!white'.format(corr=corr*100)

            graph += '\\edge [-, color={color}, line width={thickness:.15f}pt] {{{feature1}}} {{{feature2}}}; '.format(color=color, feature1=label_df.columns.values[i], feature2=label_df.columns.values[j], thickness=np.abs(corr)*2)
        
with open ('tex/correlation_graph.tex', 'w', encoding='utf-8') as f:
    f.write(graph + '\n\\end{tikzpicture}')

# Compile latex
os.system('cd tex && xelatex main.tex')

images = convert_from_path('tex/main.pdf')
for i in range(len(images)):
    width, height = images[i].size
    display(images[i].resize((2*width, 2*height)))
    images[i].save('tex/main_' + str(i) + '.jpg')

In [ ]:
# TODO: PGM fully connected graph to illustrate the complexity of
# probabilistic models, e.g. Latent Dirichlet Allocation.

In [ ]:
# TODO: Animation to the tune of 30fps LaTeX adding data points one per frame. We have 496 tweets to date which yeilds 16.5s of video. 
# Condiser music as well; do it, do it now.

In [ ]:
# TODO: Assert statements for data integrity with more_features.json.